# Claude 3 AI Chat - Interactive Interface

This notebook provides an interactive chat interface with Claude 3 AI via AWS Bedrock.

## Features:
- Real-time streaming responses
- Character counter (0-10,000 characters)
- Interactive widgets for easy use
- Error handling with helpful messages

**Prerequisites:** Run `00_setup.ipynb` first to validate your environment.

## Step 1: Import Required Modules

In [ ]:
import sys
import os
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

# Add modules directory to path
sys.path.insert(0, os.path.abspath('../modules'))

from bedrock_streaming import BedrockClient

print("✅ Modules loaded successfully")

## Step 2: Initialize Bedrock Client

In [ ]:
# Configuration
AWS_REGION = os.getenv('AWS_REGION', 'us-east-1')

# Initialize Bedrock client
bedrock = BedrockClient(region_name=AWS_REGION)

print(f"✅ Bedrock client initialized")
print(f"   Region: {AWS_REGION}")
print(f"   Model: anthropic.claude-3-sonnet-20240229-v1:0")

## Step 3: Create Interactive Chat Interface

In [ ]:
# Create widgets
prompt_input = widgets.Textarea(
    value='',
    placeholder='Ask Claude 3 anything... (e.g., "Explain quantum computing in simple terms")',
    description='Your Prompt:',
    layout=widgets.Layout(width='95%', height='120px'),
    style={'description_width': 'initial'}
)

char_counter = widgets.HTML(
    value='<p style="color: #666; font-size: 12px;">0 / 10000 characters</p>'
)

submit_btn = widgets.Button(
    description='Submit',
    button_style='primary',
    icon='paper-plane',
    layout=widgets.Layout(width='120px')
)

clear_btn = widgets.Button(
    description='Clear',
    button_style='',
    icon='trash',
    layout=widgets.Layout(width='120px')
)

response_output = widgets.Output(
    layout=widgets.Layout(
        width='95%',
        height='400px',
        border='1px solid #ddd',
        padding='15px',
        overflow_y='auto'
    )
)

status_output = widgets.HTML(
    value='<p style="color: #666; font-size: 12px;">Ready to chat with Claude 3</p>'
)

print("✅ Widgets created")

## Step 4: Define Event Handlers

In [ ]:
def update_char_counter(change):
    """Update character counter when prompt changes"""
    length = len(change['new'])
    color = '#d32f2f' if length > 10000 else '#666'
    char_counter.value = f'<p style="color: {color}; font-size: 12px;">{length} / 10000 characters</p>'
    
    # Disable submit if over limit or empty
    submit_btn.disabled = length == 0 or length > 10000

def on_submit_click(b):
    """Handle submit button click"""
    prompt = prompt_input.value.strip()
    
    if not prompt:
        with response_output:
            clear_output()
            print("⚠️  Please enter a prompt")
        return
    
    if len(prompt) > 10000:
        with response_output:
            clear_output()
            print("⚠️  Prompt exceeds 10,000 character limit")
        return
    
    # Disable buttons during processing
    submit_btn.disabled = True
    clear_btn.disabled = True
    prompt_input.disabled = True
    
    # Update status
    status_output.value = '<p style="color: #1976d2; font-size: 12px;">⏳ Claude is thinking...</p>'
    
    # Clear previous response
    with response_output:
        clear_output()
        print(f"🤖 Claude 3 Response:\n")
        print("Streaming response...\n")
    
    try:
        # Stream response
        full_response = ""
        for chunk in bedrock.invoke_streaming(prompt):
            full_response += chunk
            with response_output:
                clear_output(wait=True)
                print(f"🤖 Claude 3 Response:\n")
                print(full_response)
                print("\n[Streaming...]")
        
        # Final update
        with response_output:
            clear_output()
            print(f"🤖 Claude 3 Response:\n")
            print(full_response)
            print(f"\n---\nCompleted at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        status_output.value = f'<p style="color: #2e7d32; font-size: 12px;">✅ Response complete ({len(full_response)} characters)</p>'
    
    except Exception as e:
        with response_output:
            clear_output()
            print(f"❌ Error: {str(e)}")
            print("\nPlease check:")
            print("- Your SageMaker execution role has Bedrock permissions")
            print("- You're in a supported region (us-east-1 or us-west-2)")
            print("- Claude 3 model access is enabled in your AWS account")
        
        status_output.value = '<p style="color: #d32f2f; font-size: 12px;">❌ Error occurred</p>'
    
    finally:
        # Re-enable buttons
        submit_btn.disabled = False
        clear_btn.disabled = False
        prompt_input.disabled = False

def on_clear_click(b):
    """Handle clear button click"""
    prompt_input.value = ''
    with response_output:
        clear_output()
    status_output.value = '<p style="color: #666; font-size: 12px;">Ready to chat with Claude 3</p>'

# Attach event handlers
prompt_input.observe(update_char_counter, names='value')
submit_btn.on_click(on_submit_click)
clear_btn.on_click(on_clear_click)

print("✅ Event handlers configured")

## Step 5: Display Chat Interface

Run this cell to display the interactive chat interface:

In [ ]:
# Display header
display(Markdown("## 💬 Chat with Claude 3"))
display(Markdown("_Powered by AWS Bedrock_"))
display(Markdown("---"))

# Display prompt input section
display(Markdown("### Your Prompt"))
display(prompt_input)
display(char_counter)

# Display buttons
button_box = widgets.HBox([submit_btn, clear_btn], layout=widgets.Layout(margin='10px 0'))
display(button_box)

# Display status
display(status_output)

# Display response section
display(Markdown("### AI Response"))
display(response_output)

# Display tips
display(Markdown("""
---
### 💡 Tips:
- Enter your prompt in the text area above
- Watch the response stream in real-time
- Max prompt length: 10,000 characters
- Use Clear to reset and start a new conversation
"""))

## Example Prompts

Try these example prompts:

1. **Technical Explanation:**  
   "Explain how AWS Bedrock works in simple terms"

2. **Code Help:**  
   "Write a Python function to calculate Fibonacci numbers using recursion"

3. **Data Science:**  
   "What are the key differences between supervised and unsupervised learning?"

4. **Creative Writing:**  
   "Write a haiku about cloud computing"

5. **Analysis:**  
   "What are the pros and cons of using time-series databases like Timestream?"

## Alternative: Non-Streaming Chat

If you prefer to get the complete response at once (no streaming), use this code:

In [ ]:
# # Uncomment to use non-streaming mode
# prompt = "What is machine learning?"
# print(f"Prompt: {prompt}\n")
# print("Getting response...\n")

# response = bedrock.invoke(prompt, max_tokens=500)
# print(f"Response:\n{response}")